In [13]:
from surprise import Dataset, Reader, SVD, SVDpp, NMF
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split
import pandas as pd
from collections import defaultdict


In [2]:
import os
os.chdir(os.getcwd().replace("\\notebooks", ""))

In [83]:
student_data = pd.read_csv('rec_sys_uni/datasets/data/student/student_data.csv')
student_data['appraisal_date'] = pd.to_datetime(student_data['appraisal_date'])
student_data

,student_num,module_abb,grade,appraisal_date,pass
0,24062416,HUM3036,8.0,2018-10-20,1
1,24062416,SCI2036,7.3,2018-12-14,1
2,24076420,COR1003,7.6,2018-10-20,1
3,24076420,HUM3036,7.3,2019-03-11,1
4,24076420,HUM3040,6.4,2018-12-14,1
...,...,...,...,...,...
45137,25351684,COR1004,6.8,2023-06-02,1
45138,25351696,COR1003,6.8,2023-03-31,1
45139,25351696,COR1006,7.5,2023-03-31,1
45140,25351696,SCI2039,6.0,2023-09-05,1


In [85]:
from rec_sys_uni.datasets import datasets as ds
course_data = ds.get_course_data()
len(course_data)

149

In [86]:
column_order = list(course_data.keys())
print(list(column_order))

['COR1002', 'COR1003', 'COR1004', 'COR1006', 'HUM1003', 'HUM1007', 'HUM1010', 'HUM1011', 'HUM1012', 'HUM1013', 'HUM1016', 'HUM2003', 'HUM2005', 'HUM2007', 'HUM2008', 'HUM2013', 'HUM2016', 'HUM2018', 'HUM2021', 'HUM2022', 'HUM2030', 'HUM2031', 'HUM2046', 'HUM2047', 'HUM2051', 'HUM2054', 'HUM2056', 'HUM2057', 'HUM2058', 'HUM2059', 'HUM2060', 'HUM3014', 'HUM3019', 'HUM3029', 'HUM3034', 'HUM3036', 'HUM3040', 'HUM3042', 'HUM3043', 'HUM3044', 'HUM3045', 'HUM3049', 'HUM3050', 'HUM3051', 'HUM3052', 'HUM3053', 'SCI1004', 'SCI1005', 'SCI1009', 'SCI1010', 'SCI1016', 'SCI2002', 'SCI2009', 'SCI2010', 'SCI2011', 'SCI2017', 'SCI2018', 'SCI2019', 'SCI2022', 'SCI2031', 'SCI2033', 'SCI2034', 'SCI2035', 'SCI2036', 'SCI2037', 'SCI2039', 'SCI2040', 'SCI2041', 'SCI2042', 'SCI2043', 'SCI2044', 'SCI3003', 'SCI3005', 'SCI3006', 'SCI3007', 'SCI3046', 'SCI3049', 'SCI3050', 'SCI3051', 'SCI3052', 'SSC1005', 'SSC1007', 'SSC1025', 'SSC1027', 'SSC1029', 'SSC1030', 'SSC2002', 'SSC2004', 'SSC2006', 'SSC2007', 'SSC2008'

In [102]:
reader = Reader(rating_scale=(1.0, 10.0))

student_data_prep = student_data[['student_num', 'module_abb', 'grade']]

data = Dataset.load_from_df(student_data_prep, reader)

trainset, testset = train_test_split(data, test_size=.25, random_state=42)

algo = SVDpp(random_state=42)
algo.fit(trainset)
predictions = algo.test(testset)
evaluation = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True, return_train_measures=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7317  0.7171  0.7238  0.6930  0.7252  0.7182  0.0134  
MAE (testset)     0.5152  0.5083  0.5166  0.4963  0.5035  0.5080  0.0075  
RMSE (trainset)   0.5177  0.5151  0.5231  0.5132  0.5121  0.5162  0.0039  
MAE (trainset)    0.3758  0.3728  0.3752  0.3698  0.3697  0.3727  0.0026  
Fit time          0.71    0.71    0.71    0.70    0.71    0.71    0.00    
Test time         0.13    0.13    0.13    0.13    0.13    0.13    0.00    


In [103]:
predictions

[Prediction(uid=25134848, iid='SSC3036', r_ui=7.1, est=6.954503967441206, details={'was_impossible': False}),
 Prediction(uid=24659328, iid='SSC3019', r_ui=8.4, est=7.543995367594462, details={'was_impossible': False}),
 Prediction(uid=24765684, iid='COR1004', r_ui=6.8, est=6.285080607613009, details={'was_impossible': False}),
 Prediction(uid=24775616, iid='SSC2020', r_ui=8.0, est=8.313985191237396, details={'was_impossible': False}),
 Prediction(uid=25163380, iid='SCI2036', r_ui=7.3, est=7.5617855579594195, details={'was_impossible': False}),
 Prediction(uid=24659328, iid='COR1002', r_ui=8.2, est=7.321036564787886, details={'was_impossible': False}),
 Prediction(uid=24812384, iid='HUM2056', r_ui=7.3, est=7.2808427455921, details={'was_impossible': False}),
 Prediction(uid=25293580, iid='SSC1005', r_ui=8.7, est=8.147871749513518, details={'was_impossible': False}),
 Prediction(uid=24771692, iid='SSC1027', r_ui=8.4, est=7.928148785260162, details={'was_impossible': False}),
 Prediction

In [39]:
def get_top_n(predictions, n=5):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [40]:
top_n = get_top_n(predictions, 10)

for uid, user_ratings in top_n.items():
    print(uid, [mid for (mid, _) in user_ratings])

25277332 ['SSC2070', 'COR1006', 'SSC1007', 'SSC1005', 'SCI2034', 'SCI1009', 'SCI1009']
24749520 ['SSC3040', 'SSC3006', 'SSC2046', 'HUM2056', 'HUM2056', 'HUM2046', 'SSC2002', 'HUM1007', 'COR1004', 'HUM3040']
24829956 ['HUM2060', 'SCI3003', 'HUM2046', 'SCI2010', 'SCI1016', 'SCI2041', 'COR1003', 'HUM1012', 'SCI2002', 'COR1002']
24888724 ['HUM2016', 'HUM2016', 'SSC2064', 'HUM3053', 'HUM2056', 'SSC3047', 'SSC2002', 'SSC3032', 'SSC3032', 'HUM3050']
25277432 ['SSC2002', 'HUM1003']
24942204 ['SSC2006']
24829816 ['SSC3040', 'SSC3040', 'HUM3053', 'HUM3053', 'HUM1007', 'HUM2003', 'SSC3060', 'HUM1012', 'HUM1012', 'COR1004']
24641396 ['SSC3017', 'SSC3017', 'SSC3008', 'SSC2037', 'SSC3032', 'SSC3052', 'COR1003']
25205352 ['SSC3023', 'SSC3023', 'SSC2063', 'SSC2004', 'SSC2019']
24723204 ['SSC1030', 'SSC2008', 'SSC3017', 'SSC2063', 'SSC3006', 'SSC2018', 'COR1006', 'COR1006', 'HUM2022', 'SSC1007']
24639004 ['SSC3040', 'SSC3023', 'COR1004', 'COR1002', 'COR1002']
24728024 ['HUM2016', 'SSC2065', 'SSC3006', 